# Langroid  

We tried langroid to make agents but it turns out the Agent look constantly asks for a human input, which is a bummer.  
We will have to find another agentic framework it seems.  

In [1]:
!pip install langroid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━

In [2]:
import langroid as lr
import langroid.language_models as lm

llm_cfg = lm.OpenAIGPTConfig() # default GPT4-Turbo
mdl = lm.OpenAIGPT(llm_cfg)
mdl.chat("What is 3+4?", max_tokens=10)

REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2024-05-17 12:35:57 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


3 + 4 equals 7.

LLMResponse(message='3 + 4 equals 7.', tool_id='', function_call=None, usage=None, cached=False)

In [3]:
agent_cfg = lr.ChatAgentConfig(llm=llm_cfg)
agent = lr.ChatAgent(agent_cfg)
agent.llm_response("What is the capital of China?")
agent.llm_response("What about France?") # interprets based on previous msg

2024-05-17 12:35:58 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


The capital of China is Beijing.

Stats: N_MSG=2, TOKENS: in=13, out=7, max=1024, ctx=128000, COST: now=$0.0002, cumul=$0.0002, tot=$0.0002 (gpt-4o)

The capital of France is Paris.

Stats: N_MSG=4, TOKENS: in=24, out=7, max=1024, ctx=128000, COST: now=$0.0002, cumul=$0.0004, tot=$0.0004 (gpt-4o)

ChatDocument(content='The capital of France is Paris.', metadata=ChatDocMetaData(source='LLM', is_chunk=False, id='', window_ids=[], parent=None, sender=<Entity.LLM: 'LLM'>, tool_ids=[], parent_responder=None, block=None, sender_name='', recipient='', usage=LLMTokenUsage(prompt_tokens=24, completion_tokens=7, cost=0.00022499999999999997, calls=0), cached=False, displayed=True, status=None), function_call=None, tool_messages=[], attachment=None)

In [4]:
task = lr.Task(agent, name="Bot")
task.run("Hello")

(Bot)  User  Entity.USER(          ) (=>          ) (X           )       (          ) Hello

>>> Starting Agent Bot (1) gpt-4o 

(Bot)  User  Entity.USER(Entity.USER) (=>          ) (X           )       (          ) [CANNOT RESPOND]

(Bot)  Agent 

Hi there! How can I assist you today?

...

 q


*(Bot)  User  Entity.USER(          ) (=>          ) (X           )       (          ) q

Bye, hope this was useful!

<<< Finished Agent Bot (3) 

ChatDocument(content='q', metadata=ChatDocMetaData(source='User', is_chunk=False, id='', window_ids=[], parent=None, sender=<Entity.USER: 'User'>, tool_ids=[], parent_responder=None, block=None, sender_name='Bot', recipient='', usage=None, cached=False, displayed=False, status=<StatusCode.USER_QUIT: 'USER_QUIT'>), function_call=None, tool_messages=[], attachment=None)

In [14]:
teacher_agent = lr.ChatAgent(agent_cfg)
teacher_task = lr.Task(
  teacher_agent, name="Teacher",
  system_message="""
    Ask your student concise numbers questions, and give feedback. 
    Start with a question.
    """
)
student_agent = lr.ChatAgent(agent_cfg)
student_task = lr.Task(
  student_agent, name="Student",
  system_message="Concisely answer the teacher's questions.",
  single_round=True,
)

teacher_task.add_sub_task(student_task)
teacher_task.run()

2024-05-17 12:49:11 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2024-05-17 12:49:11 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


(Teacher)  User 

>>> Starting Agent Teacher (1) gpt-4o 

(Teacher)  User  Entity.USER(Entity.USER) (=>          ) (X           )       (          ) [CANNOT RESPOND]

(Teacher)  Agent 

...

 q


*(Teacher)  User  Entity.USER(          ) (=>          ) (X           )       (          ) q

Bye, hope this was useful!

<<< Finished Agent Teacher (2) 

ChatDocument(content='q', metadata=ChatDocMetaData(source='User', is_chunk=False, id='', window_ids=[], parent=None, sender=<Entity.USER: 'User'>, tool_ids=[], parent_responder=None, block=None, sender_name='Teacher', recipient='', usage=None, cached=False, displayed=False, status=<StatusCode.USER_QUIT: 'USER_QUIT'>), function_call=None, tool_messages=[], attachment=None)

In [15]:
from langroid.agent.chat_agent import ChatAgent, ChatAgentConfig
from langroid.agent.tools.recipient_tool import RecipientTool
from langroid.agent.task import Task
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig
from langroid.utils.configuration import set_global, Settings
from langroid.utils.logging import setup_colored_logging

student_agent = lr.ChatAgent(agent_cfg)
student_agent.enable_message(RecipientTool)
student_task = Task(
    student_agent,
    name = "Student",
    llm_delegate=True,
    single_round=False,
    system_message="""
    Your task is to write 4 short bullet points about 
    Language Models in the context of Machine Learning (ML),
    especially about training, and evaluating them. 
    However you are a novice to this field, and know nothing about this topic. 
    To collect your bullet points, you will consult 2 people:
    TrainingExpert and EvaluationExpert.
    You will ask ONE question at a time, to ONE of these experts. 
    To clarify who your question is for, you must use 
    the `recipient_message` tool/function-call, setting 
    the `content` field to the question you want to ask, and the
    `recipient` field to either TrainingExpert or EvaluationExpert.

    Once you have collected the points you need,
    say DONE, and show me the 4 bullet points. 
    """,
)
training_expert_agent = lr.ChatAgent(agent_cfg)
training_expert_task = Task(
    training_expert_agent,
    name = "TrainingExpert",
    system_message="""
    You are an expert on Training Language Models in Machine Learning. 
    You will receive questions on this topic, and you must answer these
    very concisely, in one or two sentences, in a way that is easy for a novice to 
    understand.
    """,
    single_round=True,  # task done after 1 step() with valid response
)

evaluation_expert_agent = lr.ChatAgent(agent_cfg)
evaluation_expert_task = Task(
    evaluation_expert_agent,
    name = "EvaluationExpert",
    system_message="""
    You are an expert on Evaluating Language Models in Machine Learning. 
    You will receive questions on this topic, and you must answer these
    very concisely, in one or two sentences, in a way that is easy for a novice to 
    understand.
    """,
    single_round=True,  # task done after 1 step() with valid response
)

student_task.add_sub_task(
    [training_expert_task, evaluation_expert_task]
)
student_task.run()

2024-05-17 13:02:39 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2024-05-17 13:02:40 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client
2024-05-17 13:02:40 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


(Student)  User 

>>> Starting Agent Student (1) gpt-4o 

(Student)  User  Entity.USER(Entity.USER) (=>          ) (X           )       (          ) [CANNOT RESPOND]

(Student)  Agent 

...

 q


*(Student)  User  Entity.USER(          ) (=>          ) (X           )       (          ) q

Bye, hope this was useful!

<<< Finished Agent Student (10) 

ChatDocument(content='q', metadata=ChatDocMetaData(source='User', is_chunk=False, id='', window_ids=[], parent=None, sender=<Entity.USER: 'User'>, tool_ids=[], parent_responder=None, block=None, sender_name='Student', recipient='', usage=None, cached=False, displayed=False, status=<StatusCode.USER_QUIT: 'USER_QUIT'>), function_call=None, tool_messages=[], attachment=None)